Right now I just want to read the (currently) preprocessed data and start to set up a workflow for each event calculating the statistics for one station. Later, we can add additional stations into the framework and incorporate the rest of the data.

In [1]:
## import libraries
import os, sys
import yaml
import xarray as xr
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units

%matplotlib inline

sys.path.append('../modules')
import ar_funcs

In [2]:
path_to_data = '/cw3e/mead/projects/cwp140/scratch/dnash/data/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
ARID = 201508151811

#PASI Sitka 57.04800 -135.36470
stationID = 'PASI'

## at the end, we want a dataframe for each station in Aaron's database
##TODO READ IN station.txt and iterate through all stations
# load in station.txt, read name, lat, lon, code
#PASI Sitka 57.04800 -135.36470
# station_code = PASI
# station_name = 'Sitka'
# station_lat = 57.04800
# station_lon = -135.36470 % 360

df_fname = 'df_{0}'.format(stationID)

In [4]:
%%time

## TODO: iterate through all ARIDs

## step 1 - read duration df and create a copy with empty columns
df = ar_funcs.build_empty_df(stationID)

## step 2 - for each ARID, pull in IVT data
df = ar_funcs.preprocess_IVT_info(stationID, ARID, df)

## step 3 - for each ARID, pull in freezing level data
df = ar_funcs.preprocess_freezing_level(stationID, ARID, df)

## step 4 - for each ARID, pull in GEFS precipitation data
df = ar_funcs.preprocess_prec_GEFS(stationID, ARID, df)

## step 5 - for each ARID, pull in ASOS precipitation data and ARI

## step 6 - for each ARID, pull in Aaron's impact information


df.loc[ARID]

CPU times: user 834 ms, sys: 138 ms, total: 972 ms
Wall time: 1.05 s


Unnamed: 0                                     1203
trackID                              201508151811.0
start_date                      2015-08-17 06:00:00
end_date                        2015-08-19 06:00:00
duration                                       48.0
IVT_max                                  642.900439
IVT_max_time          2015-08-18T15:00:00.000000000
IVT_dir                                    4.246507
freezing_level                          3512.039062
ar_scale                                        2.0
GFS_prec_accum                            36.639999
GFS_prec_max_rate                              19.1
ASOS_prec_accum                                 NaN
ASOS_prec_max_rate                              NaN
ARI_1hr                                         NaN
ARI_3hr                                         NaN
ARI_6hr                                         NaN
ARI_12hr                                        NaN
ARI_24hr                                        NaN
impact_scale